In [1]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = ""

In [2]:
! pip install -qU langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI

llm= ChatOpenAI(
    model="gpt-4o",
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key= os.environ["OPENAI_API_KEY"]
)

In [4]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7da486afaaa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7da4866eaa70>, root_client=<openai.OpenAI object at 0x7da486de70d0>, root_async_client=<openai.AsyncOpenAI object at 0x7da486afa7a0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), max_retries=2)

# 체인 사용 x

In [5]:
prompt = "서울의 수도는 ?"

In [6]:
result = llm.invoke(prompt)

In [7]:
result.content

'서울은 대한민국의 수도입니다.'

# 질문 구조화

In [8]:
prompt_2 = [
    ("system", "너는 제시된 국가의 수도가 어디인지 알려주는 어시스턴트야. 결과는 단어로 제시"),
    ("human","중국")
]

In [10]:
result_2 = llm.invoke(prompt_2)

In [13]:
result_2.content

'베이징'

# PromptTemplate

In [19]:
from langchain.prompts import PromptTemplate

template = "{input}의 수도는 어디인가요?"

prompt_3 = PromptTemplate.from_template(template)
prompt_3

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}의 수도는 어디인가요?')

In [20]:
prompt_3 = prompt_3.format(input="중국")
prompt_3

'중국의 수도는 어디인가요?'

In [21]:
result_3 = llm.invoke(prompt_3)
result_3.content

'중국의 수도는 베이징(北京)입니다.'

#chain

In [23]:
template = "{input}의 수도는 어디인가요?"

prompt_4 = PromptTemplate.from_template(template)

chain = prompt_4 | llm

In [24]:
result_4 = chain.invoke({"input":"중국"})
result_4.content

'중국의 수도는 베이징입니다.'

# PromptTemplate 객체를 생성하며, template 문자열과 input_variables를 동시에 지정

In [27]:
from langchain_core.prompts import PromptTemplate

template = "{input_1},{input_2},{input_3}의 역할"

prompt_5 = PromptTemplate(
    template=template,
    input_variables=["input_1","input_2","input_3"],
    partial_variables={"input_2": "GPU","input_3": "RAM"}
)

print(prompt_5)

input_variables=['input_1', 'input_2', 'input_3'] input_types={} partial_variables={} template='{input_1},{input_2},{input_3}의 역할'


In [28]:
chain = prompt_5 | llm

In [29]:
result_5 = chain.invoke({"input_1":"CPU","input_2":"GPU","input_3":"RAM"})
result_5.content

'컴퓨터 시스템에서 CPU, GPU, RAM은 각각 특정한 역할을 수행하며, 시스템의 전반적인 성능과 기능에 큰 영향을 미칩니다.\n\n1. **CPU (Central Processing Unit, 중앙 처리 장치)**:\n   - **역할**: CPU는 컴퓨터의 두뇌 역할을 하며, 프로그램 명령을 해석하고 실행합니다. 수학적 계산, 데이터 처리, 프로그램 제어 흐름 등을 담당합니다.\n   - **구성 요소**: ALU(산술 논리 장치), CU(제어 장치), 레지스터 등으로 구성됩니다.\n   - **고려 사항**: CPU 성능은 주파수(클록 속도), 코어 수, 스레드 수 등에 따라 달라지며, 다중 작업 처리와 컴퓨터의 전반적인 응답 속도에 큰 영향을 미칩니다.\n\n2. **GPU (Graphics Processing Unit, 그래픽 처리 장치)**:\n   - **역할**: GPU는 주로 그래픽 및 영상 처리에 특화된 프로세서입니다. 복잡한 3D 그래픽 렌더링, 고해상도 이미지 및 비디오 처리, 머신러닝 작업을 가속화하는 데 사용됩니다.\n   - **고려 사항**: GPU는 병렬 처리 능력이 뛰어나 많은 코어를 사용하여 작업을 동시에 처리할 수 있습니다. 따라서 그래픽 요구 사항이 높은 게임이나 과학적 시뮬레이션, 딥러닝 등에 유리합니다.\n\n3. **RAM (Random Access Memory, 임의 접근 메모리)**:\n   - **역할**: RAM은 컴퓨터의 휘발성 메모리로, 현재 실행 중인 프로그램과 사용 중인 데이터를 저장합니다. CPU가 정보를 빠르게 읽고 쓸 수 있게 하여 시스템 성능을 향상시킵니다.\n   - **특징**: RAM은 전원이 끊기면 저장된 데이터가 사라지며, 속도가 빠르고 접근 시간이 짧습니다.\n   - **고려 사항**: RAM의 용량은 동시에 실행할 수 있는 프로그램 수와 데이터 처리량에 영향을 미칩니다. 용량이 부족하면 시스템이 느려질 수 있습니다.\n\n이 세 가지 구성 요소는 모두 컴퓨터 성능에 중요한 역할을

___

In [32]:
from datetime import datetime

prompt_6 = PromptTemplate(
    template="오늘 날짜는 {today} 입니다. 오늘 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={"today" : datetime.now().strftime("%Y %B %d")},
)

In [33]:
prompt_6

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': '2025 January 13'}, template='오늘 날짜는 {today} 입니다. 오늘 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.')

In [34]:
prompt_6.format(n=3)

'오늘 날짜는 2025 January 13 입니다. 오늘 생일인 유명인 3명을 나열해 주세요. 생년월일을 표기해주세요.'

In [35]:
chain  = prompt_6 | llm

In [37]:
result_6 = chain.invoke({"n":3}).content

In [38]:
result_6

'오늘 날짜가 2025년 1월 13일이라면, 생일을 맞이하는 몇몇 유명인을 아래에 예시로 들어 드릴 수 있습니다. 그러나 항상 정확성과 최신 정보를 확인하기 위해 인터넷이나 신뢰할 수 있는 출처를 참조하세요.\n\n1. 패트릭 드와이어 (Patrick Dwyer) - 1992년 1월 13일\n2. 나탈리아 다이어 (Natalia Dyer) - 1995년 1월 13일\n3. 리암 헴스워스 (Liam Hemsworth) - 1990년 1월 13일\n\n이 리스트는 완벽하지 않을 수 있으며, 특정 유명인의 생일은 자료마다 다르게 기재될 수 있습니다. 추가적인 검증을 권장합니다.'

___

In [40]:
from langchain.prompts import load_prompt

prompt_7 = load_prompt("/content/fruit_color.yaml", encoding="UTF-8")
prompt_7

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [42]:
chain = prompt_7 | llm

In [43]:
chain

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7da486afaaa0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7da4866eaa70>, root_client=<openai.OpenAI object at 0x7da486de70d0>, root_async_client=<openai.AsyncOpenAI object at 0x7da486afa7a0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), max_retries=2)

In [50]:
prompt_7 = prompt_7.format(fruit = "망고")
prompt_7

'망고의 색깔이 뭐야?'

In [53]:
result_7_1 = llm.invoke(prompt_7).content
result_7_1

'망고의 색깔은 익은 정도에 따라 달라질 수 있습니다. 익기 전의 망고는 보통 녹색을 띠며, 익어가면서 노란색, 주황색, 또는 붉은 색을 띌 수 있습니다. 완전히 익은 망고는 주로 노란색이나 주황빛을 많이 보입니다. 망고의 종류에 따라서도 색깔에 약간의 차이가 있을 수 있습니다.'

In [54]:
result_7_2 = chain.invoke(prompt_7).content
result_7_2

'망고의 색깔은 주로 노란색, 주황색, 빨간색, 혹은 녹색 등의 조합으로 이루어진 경우가 많습니다. 망고가 익을수록 색깔이 더 강렬해지는 경우가 일반적입니다. 하지만 망고의 종류에 따라 색깔은 다를 수 있습니다.'